In [1]:
import requests, os, time
import pandas as pd
import datetime as dt
import numpy as np
from datetime import date, timedelta
from bs4 import BeautifulSoup
from dotenv import load_dotenv
load_dotenv()
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', None)

In [ ]:
def player_database():
    
    start = time.time()
    
    p_database = p_database.lower()

    abcd = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    player_url = "https://www.pro-football-reference.com/players/{abcd}/"
    players = []
    for c in range(0,len(abcd)):
        res = requests.get(player_url.format(abcd=abcd[c]))
        soup = BeautifulSoup(res.content, 'html.parser')
        data = soup.find_all('p')

        for i in data:
            a = i.find('a')['href']
            pl = i.get_text()[:(i.get_text().find("(")-1)]
            po = i.get_text()[(i.get_text().find("(")+1):i.get_text().find(")")]
            fy = i.get_text()[(i.get_text().rfind("-")-4):(i.get_text().rfind("-"))]
            ly = i.get_text()[(i.get_text().rfind("-")+1):(i.get_text().rfind("-")+5)]
            p = {"Address": a[:-4], 
                 "Player": pl, 
                 "Position": po, 
                 "First_Year":  fy, 
                 "Last_Year": ly}
            players.append(p)

    players_df = pd.DataFrame().from_dict(players)
    players_df = players_df[players_df.Last_Year != "Ever"]
    players_df['Player'] = players_df['Player'].str.replace('+','')
    players_df['Last_Year'] = players_df.Last_Year.astype('int32')
    players_df.to_csv('../data/database-players.csv',index=False)

In [ ]:
def team_data(s,url):
    # scrape weekly team stats from www.stathead.com
    start = time.time()
    # page to start scrape at
    page = 0
    # login payload information pulled from a .env file
    stat_login_url = "https://stathead.com/users/login.cgi"
    stat_user_name = os.environ.get('statheadusername')
    stat_password = os.environ.get('statheadpassword')
    stat_payload = {
        'username': stat_user_name,
        'password': stat_password
    }
    
    # lots of team data, could not fit into one pull from stathead, needed to use 2 different url's, this allows the function to iterate through both
    if url == 1:
        stat_url = 'https://stathead.com/football/tgl_finder.cgi?request=1&temperature_gtlt=lt&game_num_max=99&week_num_max=99&order_by=points&match=game&year_max={Season}&order_by_asc=0&week_num_min=0&game_type=E&game_num_min=0&year_min={Season}&cstat[1]=all_td_team&ccomp[1]=gt&cval[1]=0&cstat[2]=third_down_att&ccomp[2]=gt&cval[2]=0&cstat[3]=vegas_line&ccomp[3]=gt&cval[3]=-50&cstat[4]=penalties&ccomp[4]=gt&cval[4]=0&cstat[5]=rush_att&ccomp[5]=gt&cval[5]=0&cstat[6]=tot_yds&ccomp[6]=gt&cval[6]=0&cstat[7]=first_down&ccomp[7]=gt&cval[7]=0&cstat[8]=punt&ccomp[8]=gt&cval[8]=0&cstat[9]=pass_cmp&ccomp[9]=gt&cval[9]=0&offset={page}'
    elif url == 2:
        stat_url = 'https://stathead.com/football/tgl_finder.cgi?request=1&temperature_gtlt=lt&game_num_max=99&week_num_max=99&order_by=all_td_opp&match=game&year_max={Season}&order_by_asc=0&week_num_min=0&game_type=R&game_num_min=0&year_min={Season}&cstat[1]=tot_yds_opp&ccomp[1]=gt&cval[1]=0&cstat[2]=rush_yds_diff&ccomp[2]=gt&cval[2]=-500&cstat[3]=score_diff_thru_1&ccomp[3]=gt&cval[3]=-500&cstat[4]=rush_att_opp&ccomp[4]=gt&cval[4]=0&cstat[5]=kick_ret_td_tgl&ccomp[5]=gt&cval[5]=0&cstat[6]=pass_cmp_opp&ccomp[6]=gt&cval[6]=0&cstat[7]=first_down_opp&ccomp[7]=gt&cval[7]=0&cstat[8]=score_diff_1_qtr&ccomp[8]=gt&cval[8]=-500&cstat[9]=third_down_att_opp&ccomp[9]=gt&cval[9]=0&offset={page}'
    elif url != 1 or 2:
        print("Please select 1 or 2.")
    
    # open logged in session for scraping
    with requests.Session() as session:

        s = session.post(stat_login_url, data=stat_payload)
        
        # beginning the scrape and stopping the scrape when page number reaches 100k
        try:

            while page < 100000:
                
                # pulling the website and scraping it
                website = session.get(stat_url.format(Season=s,page=page)).text
                soup = BeautifulSoup(website, 'html')
                table = soup.find('table', attrs={'class': 'sortable', 'id': 'results'})
                
                # pull headers and rows out of the data
                table_headers = [header.text for header in table.find('thead').find_all('th')]
                table_rows = table.find_all('tr')
                
                # final location for complete data
                final_data = []
                
                # create row for each line of data in table
                for tr in table_rows:
                    td = tr.find_all('td')
                    row = [tr.text for tr in td]
                    final_data.append(row)
                
                # create the dataframe in panadas excluding the blank row and matching headers with the rows
                df = pd.DataFrame(final_data[1:], columns=table_headers[12:])
                
                # writting dataframe to csv, continuous appending just incase the function fails, data will be saved
                if url == 1:
                    df.to_csv(f'../data/raw-data/nfl-team-data-{s}-1-raw.csv',mode='a',index=False)
                else:
                    df.to_csv(f'../data/raw-data/nfl-team-data-{s}-2-raw.csv',mode='a',index=False)
                
                # progress through the websites
                page += 100
            
        except:
            # notifying the scrape has completed.
            end = time.time()
            print(f'Done: Team {s}, {url}, {page}',f'Time to complete: {end-start}',sep='\n')

In [ ]:
def injury_reports(s):

    # scrape weekly nfl injury reports
    start = time.time()
    # list of team abbreviations from pro-football-reference for url purposes
    teams = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den', 'det', 'gnb','htx','clt','jax','kan',
             'sdg','ram','mia','min','nor','nwe','nyg','nyj','rai','phi','pit','sea','sfo','tam','oti','was']

    ENDPOINT = 'https://www.pro-football-reference.com/teams/{team}/{Season}_injuries.htm'
    
    # creating final place to store data
    final_df = pd.DataFrame()

    # scrape pages for all teams
    for team in teams:
        
        # open webpage and scrape contents into lists, then to a dataframe
        res = requests.get(ENDPOINT.format(Season=s,team=team))

        soup = BeautifulSoup(res.content, 'lxml')

        table = soup.find('table', attrs={'class': 'sortable', 'id': 'team_injuries'})
        table_rows = table.find_all('tr')

        final_data = []
        for tr in table_rows:
            td = tr.find_all(['th','td'])
            row = [tr['data-tip'] if tr.has_attr("data-tip") else tr.text for tr in td]
            final_data.append(row)

        df_data = final_data[1:]
        data_body = [[df_data[j][i] for j in range(len(df_data))] for i in range(len(df_data[0]))]

        df = pd.DataFrame(data_body,final_data[0]).T
        # adding team and season columns for identification
        df.insert(loc=1,column='Team',value=team)
        df.insert(loc=2,column='Season',value=s)

        # combine current data with final dataframe
        final_df = pd.concat([final_df, df])
    
    # rename column
    final_df.rename(columns={'PlayerÂ ':'Player'},inplace=True)
    
    # write final data to csv file
    final_df.to_csv(f'../data/raw-data/nfl-injury-report-{s}-raw.csv',index=False)
    end = time.time()
    print(f'Done: Injury Reports {s}',f'Time to complete: {end-start}',sep='\n')

In [5]:
def player_stats(s):
    start = time.time()
    players = pd.read_csv('../data/database-players.csv')

    final_df = pd.DataFrame()

    for p_add in players['Address']:

        try:
            p_id = p_add[11:]
            df = pd.read_html(f'https://www.pro-football-reference.com{p_add}/gamelog/{s}', header=[0,1], attrs={'id': 'stats'})
            df = df[0]

            df['Player_Address'] = p_add
            df['Season'] = s

            final_df = pd.concat([df, final_df])
            print(p_add)

        except:
            continue

    final_df.to_csv(f'../data/data-raw/player-weekly-stats-{s}-raw.csv', index=False)
    end = time.time()
    print(f'Done: Player stats scrape (including snap counts) for {s} season',f'{(end-start)/60}: minutes to complete.', sep='\n')

In [ ]:
player_stats(2020)

/players/A/AbduAm00
/players/A/AbouOd00
/players/A/AbraJo01
/players/A/AbraDe00
/players/A/AdamAn00
/players/A/AdamDa01
/players/A/AdamJa00
/players/A/AdamJo03
/players/A/AdamMa00
/players/A/AdamMo00
/players/A/AdamTy00
/players/A/AddaJa00
/players/A/AddeNa00
/players/A/AddiMa00
/players/A/AdenHa00
/players/A/AdenOl00
/players/A/AdeoAa00
/players/A/AghoNe00
/players/A/AgimMc00
/players/A/AgneJa00
/players/A/AhmeSa01
/players/A/AiyuBr00
/players/A/AkerCa00
/players/A/AkinFr01
/players/A/AkinJo00
/players/A/AlShAz00
/players/A/AlakOt01
/players/A/AlexJa00
/players/A/AlexKw00
/players/A/AlexMa01
/players/A/AlieMo00
/players/A/AlleNi00
/players/A/AlleBr00
/players/A/AlleBr01
/players/A/AlleBr02
/players/A/AlleDa01
/players/A/AlleJo01
/players/A/AlleJo02
/players/A/AlleJo03
/players/A/AlleKe00
/players/A/AlleKy00
/players/A/AlleMa03
/players/A/AlleRi00
/players/A/AlleRy01
/players/A/AlleZa01
/players/A/AluaTy99
/players/A/AlufAu00
/players/A/AmadUg00
/players/A/AmenDa00
/players/A/AmosAd00
